# Demo: Schilder-classificatie met Gradio (VGG16 finetuning)

In deze notebook bouwen we een **kleine demo-app** (met **Gradio**) waarmee een gebruiker:

- een willekeurige afbeelding uploadt;
- de **voorspelde schilder** ziet;
- én de **probabiliteiten per klasse** (softmax-scores).

## Verwachte mappenstructuur (zoals in jouw project)

```
OPDRACHT SCHILDERIJEN CLASSIFICEREN/
│
├── datasets/
│   └── alle_schilders/
│       ├── train/
│       ├── val/
│       └── test/
│
└── notebooks/
    └── alle schilders/
        ├── demo_gradio_alle_schilders.ipynb   ← deze notebook
        └── vgg16_finetuning/
            ├── best_model_phase2.keras        ← beste finetuned model
            └── ...
```

> **Belangrijk:** we gebruiken enkel `train/` om de **klassen-namen** (labels) in de juiste alfabetische volgorde te lezen.


## 1) Imports + paden instellen

We bepalen hier **robust** de paden:

- `DATASET_DIR` wijst naar `datasets/alle_schilders/` (twee niveaus hoger dan deze notebook)
- `MODEL_PATH` wijst naar het beste finetuning model: `vgg16_finetuning/best_model_phase2.keras`


In [12]:
from pathlib import Path
import numpy as np


from tensorflow import keras
from tensorflow.keras.applications.vgg16 import preprocess_input

import gradio as gr
from PIL import Image

# Paden (RELATIEF, robust)
NB_DIR = Path.cwd()  # notebooks/alle schilders

# datasets/alle_schilders (twee niveaus hoger dan NB_DIR)
DATASET_DIR = NB_DIR.parents[1] / "datasets" / "alle_schilders"
TRAIN_DIR = DATASET_DIR / "train"
VAL_DIR   = DATASET_DIR / "val"
TEST_DIR  = DATASET_DIR / "test"

# Beste finetuning model (Phase 2)
MODEL_PATH = NB_DIR / "vgg16_finetuning" / "best_model_phase2.h5"

print("NB_DIR      :", NB_DIR)
print("DATASET_DIR :", DATASET_DIR)
print("MODEL_PATH  :", MODEL_PATH)

# Snelle checks (handig bij path-problemen)
assert TRAIN_DIR.exists(), f"TRAIN_DIR niet gevonden: {TRAIN_DIR}"
assert VAL_DIR.exists(),   f"VAL_DIR niet gevonden: {VAL_DIR}"
assert TEST_DIR.exists(),  f"TEST_DIR niet gevonden: {TEST_DIR}"
assert MODEL_PATH.exists(), f"MODEL_PATH niet gevonden: {MODEL_PATH}"

print("Paden zijn correct.")

NB_DIR      : c:\Users\Moustafa\Documents\deep learning\opdracht schilderijen classificeren\notebooks\alle schilders
DATASET_DIR : c:\Users\Moustafa\Documents\deep learning\opdracht schilderijen classificeren\datasets\alle_schilders
MODEL_PATH  : c:\Users\Moustafa\Documents\deep learning\opdracht schilderijen classificeren\notebooks\alle schilders\vgg16_finetuning\best_model_phase2.h5
Paden zijn correct.


In [13]:
import os
p = r"vgg16_finetuning\best_model_phase2.h5"
print("bytes:", os.path.getsize(p))
print("MB:", os.path.getsize(p)/1024/1024)

bytes: 59442232
MB: 56.68852996826172


## 3) Klassen-namen ophalen (labels)

`image_dataset_from_directory` gebruikt standaard **alfabetische volgorde** van foldernamen.  
Daarom lezen we de klassen ook alfabetisch in uit `train/`.


In [2]:
# Klassen-namen (alphabetisch)
class_names = sorted([p.name for p in TRAIN_DIR.iterdir() if p.is_dir()])
num_classes = len(class_names)

print("Classes:", class_names)
print("Num classes:", num_classes)

Classes: ['Mondriaan', 'Picasso', 'Rembrandt', 'Rubens']
Num classes: 4


## 4) Model laden

We laden het getrainde `.keras` model van finetuning phase 2.


In [14]:
model = keras.models.load_model(MODEL_PATH, compile=False)
print("Model geladen")

Model geladen


## 5) Predict-functie

- Gradio levert een **PIL image**.
- We resizen naar **224×224** (VGG16 input).
- We passen `preprocess_input` toe (VGG16 verwacht BGR + mean subtraction).
- We tonen:
  - de **top-1 voorspelling**
  - de **top-1 probabiliteit**
  - alle probabilities als dictionary (Gradio kan dit mooi tonen)


In [15]:
IMG_SIZE = (224, 224)

def predict_painter(img: Image.Image):
    if img is None:
        return "Geen afbeelding", 0.0, {}

    # Zorg dat de input RGB is
    img = img.convert("RGB")

    # Resize naar VGG16 input size
    img_resized = img.resize(IMG_SIZE)

    # Naar numpy + batch dimension
    x = np.array(img_resized, dtype=np.float32)
    x = np.expand_dims(x, axis=0)

    # VGG16 preprocessing
    x = preprocess_input(x)

    # Predict (softmax probabilities)
    probs = model.predict(x, verbose=0)[0]  # shape (num_classes,)
    top_idx = int(np.argmax(probs))
    top_label = class_names[top_idx]
    top_prob = float(probs[top_idx])

    prob_dict = {class_names[i]: float(probs[i]) for i in range(num_classes)}

    return top_label, top_prob, prob_dict

## 6) Gradio interface

Klik op **"Voorspel"** na het uploaden van een afbeelding.  
Je krijgt:
- de voorspelde schilder,
- de top-1 probabiliteit,
- en een overzicht van alle probabilities.


In [16]:
with gr.Blocks() as demo:
    gr.Markdown(
        "#Schilder-classificatie (VGG16 finetuning"
        "Upload een schilderij en krijg de voorspelde schilder + probabilities."
    )

    with gr.Row():
        inp = gr.Image(type="pil", label="Upload een afbeelding")
        with gr.Column():
            out_label = gr.Textbox(label="Voorspelde schilder")
            out_prob  = gr.Number(label="Probabiliteit (top-1)")
            out_probs = gr.Label(num_top_classes=num_classes, label="Probabilities per schilder")

    btn = gr.Button("Voorspel")
    btn.click(fn=predict_painter, inputs=inp, outputs=[out_label, out_prob, out_probs])

demo

Gradio Blocks instance: 1 backend functions
-------------------------------------------
fn_index=0
 inputs:
 |-<gradio.components.image.Image object at 0x000001A762196FD0>
 outputs:
 |-<gradio.components.textbox.Textbox object at 0x000001A762368C90>
 |-<gradio.components.number.Number object at 0x000001A76235CAD0>
 |-<gradio.components.label.Label object at 0x000001A7599DC050>

## 7) Start de app

- In Jupyter/VS Code Notebook: dit opent meestal een link in je output.
- Lokaal werkt CPU prima (GPU is niet nodig voor inference).


In [17]:
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
